What is in here?

$\textbf{Lasso on shift interventional data using numpy}$ for full graphs with 2, 3, 4, and 5 nodes.

In the case of 2 nodes, different rows from $A$ have been deleted but also the whole matrix has been used for the estimation using Lasso.

In the case of 3 nodes, different entries of $M$ have been set to zero to simulate subgraphs.

## Preparation

In [1]:
import numpy as np
from sympy import *
# from sympy.solvers import solve
import warnings
from scipy.linalg import solve_continuous_lyapunov
from sklearn import linear_model
from sklearn.exceptions import ConvergenceWarning
warnings.filterwarnings('ignore', category=ConvergenceWarning) 
init_printing(use_unicode=True)

In [2]:
def simulate_M(seed: int, p: int) -> Array:
    np.random.seed(seed=seed)
    normal_matrix = np.random.normal(0, 1, (p, p))
    for i in range(p): # adjust diagonal entries s.t. M stable
        normal_matrix[i, i] = - np.sum(np.abs(normal_matrix[i, :]))
    return normal_matrix

def num_sym(p: int) -> int:
    return int(p * (p + 1) / 2)

def create_A_Sigma(cov: Array, p: int) -> Array:
    row_num = num_sym(p=p)
    col_num = int(p * p)
    A_Sigma = np.empty(shape=(row_num, col_num), dtype=float)
    for l in range(p):
        for k in range(l+1):
            for i in range(p):
                for j in range(p):
                    if (j != k) & (j != l):
                        A_Sigma[(k * p) + (l - num_sym(p=k)), i * p + j] = 0
                    elif (j == k) & (k != l):
                        A_Sigma[(k * p) + (l - num_sym(p=k)), i * p + j] = cov[l, i]
                    elif (j == l) & (l != k):
                        A_Sigma[(k * p) + (l - num_sym(p=k)), i * p + j] = cov[k, i]
                    elif (j == k) & (j == l):
                        A_Sigma[(k * p) + (l - num_sym(p=k)), i * p + j] = 2 * cov[j, i]
    return A_Sigma

def create_A(A_Sigma: Array, mean: Array, p: int) -> Array:
    identity = mean[0] * eye(p)
    for i in range(1, p):
        identity = np.hstack((identity, mean[i] * eye(p)))
    return np.vstack((A_Sigma, identity))

In [3]:
num_exp = 1000 # number of true matrices per case
num_sim = 1000 # number of simulations per true matrix

index_intervention = 0
b = 2

## 2 nodes

In [4]:
p = 2

C = 2 * np.eye(p)
vech_C = np.array([C[i, j] for i in range(p) for j in range(i, p)])
unit_v = np.eye(p)[:, index_intervention]

reg_param = [0.0001, 0.001, 0.01, 0.1, 1]
num_param = len(reg_param) + 1 # also linear regression

### $\texttt{index\_deletion = 0}$
***Results:*** 
* Average Frobenius distance between the estimation $\hat{M}$ and the true matrix $M^*$ for linear regression is $\large{0.815}$.
* Average Frobenius distance between the estimation $\hat{M}$ and the true matrix $M^*$ for Lasso regression with $\lambda = 0.0001$ is $\large{1.033}$.
* Average Frobenius distance between the estimation $\hat{M}$ and the true matrix $M^*$ for Lasso regression with $\lambda = 0.001$ is $\large{0.965}$.
* Average Frobenius distance between the estimation $\hat{M}$ and the true matrix $M^*$ for Lasso regression with $\lambda = 0.01$ is $\large{0.966}$.
* Average Frobenius distance between the estimation $\hat{M}$ and the true matrix $M^*$ for Lasso regression with $\lambda = 0.1$ is $\large{1.498}$.
* Average Frobenius distance between the estimation $\hat{M}$ and the true matrix $M^*$ for Lasso regression with $\lambda = 1$ is $\large{2.645}$.

In [5]:
index_deletion = 0
offset = 1

c = np.hstack((vech_C, - b * unit_v))
c = np.delete(c, (index_deletion))

estimate_2_0 = np.empty(shape=(num_exp, num_param), dtype=object)
frobnorm_2_0 = np.empty(shape=(num_exp, num_param), dtype=object)

for seed in range(offset, num_exp + offset):
    true_M = simulate_M(seed=seed, p=p)

    true_cov = solve_continuous_lyapunov(a=true_M, q=-C)
    true_mean = (b * np.linalg.inv(a=true_M)).dot(unit_v)

    obs = np.random.multivariate_normal(mean = true_mean, cov = true_cov, size = num_sim)
    emp_cov = np.cov(obs, rowvar=False)
    emp_mean = np.mean(obs, axis=0)

    emp_A_Sigma = create_A_Sigma(cov=emp_cov, p=p)
    emp_A = np.delete(create_A(A_Sigma=emp_A_Sigma, mean=emp_mean, p=p), (index_deletion), axis=0)

    # Linear regression
    lin_reg = linear_model.LinearRegression()
    lin_reg.fit(X=emp_A, y=-c)
    est_M = np.transpose(np.reshape(lin_reg.coef_, shape=(p, p)))
    frob = np.linalg.norm(est_M - true_M)

    estimate_2_0[seed-offset, 0] = est_M.round(3)
    frobnorm_2_0[seed-offset, 0] = frob

    for index, param in enumerate(reg_param):
        direct_lasso = linear_model.Lasso(alpha=param, max_iter=5000)
        direct_lasso.fit(X=emp_A, y=-c)
        est_M = np.transpose(np.reshape(direct_lasso.coef_, shape=(p, p)))
        frob = np.linalg.norm(est_M - true_M)

        estimate_2_0[seed-offset, index+1] = est_M.round(3)
        frobnorm_2_0[seed-offset, index+1] = frob

In [6]:
np.mean(frobnorm_2_0, axis=0)

array([np.float64(0.8149207435168655), np.float64(1.0325061959683624),
       np.float64(0.9651368666292439), np.float64(0.9656801934246148),
       np.float64(1.4977554525252295), np.float64(2.6449216219550697)],
      dtype=object)

### $\texttt{index\_deletion = 1}$
***Results:*** 
* Average Frobenius distance between the estimation $\hat{M}$ and the true matrix $M^*$ for linear regression is $\large{1.11}$.
* Average Frobenius distance between the estimation $\hat{M}$ and the true matrix $M^*$ for Lasso regression with $\lambda = 0.0001$ is $\large{1.023}$.
* Average Frobenius distance between the estimation $\hat{M}$ and the true matrix $M^*$ for Lasso regression with $\lambda = 0.001$ is $\large{1.098}$.
* Average Frobenius distance between the estimation $\hat{M}$ and the true matrix $M^*$ for Lasso regression with $\lambda = 0.01$ is $\large{1.126}$.
* Average Frobenius distance between the estimation $\hat{M}$ and the true matrix $M^*$ for Lasso regression with $\lambda = 0.1$ is $\large{1.534}$.
* Average Frobenius distance between the estimation $\hat{M}$ and the true matrix $M^*$ for Lasso regression with $\lambda = 1$ is $\large{2.545}$.

In [7]:
index_deletion = 1
offset = 12

c = np.hstack((vech_C, - b * unit_v))
c = np.delete(c, (index_deletion))

estimate_2_1 = np.empty(shape=(num_exp, num_param), dtype=object)
frobnorm_2_1 = np.empty(shape=(num_exp, num_param), dtype=object)

for seed in range(offset, num_exp + offset):
    true_M = simulate_M(seed=seed, p=p)

    true_cov = solve_continuous_lyapunov(a=true_M, q=-C)
    true_mean = (b * np.linalg.inv(a=true_M)).dot(unit_v)

    obs = np.random.multivariate_normal(mean = true_mean, cov = true_cov, size = num_sim)
    emp_cov = np.cov(obs, rowvar=False)
    emp_mean = np.mean(obs, axis=0)

    emp_A_Sigma = create_A_Sigma(cov=emp_cov, p=p)
    emp_A = np.delete(create_A(A_Sigma=emp_A_Sigma, mean=emp_mean, p=p), (index_deletion), axis=0)

    # Linear regression
    lin_reg = linear_model.LinearRegression()
    lin_reg.fit(X=emp_A, y=-c)
    est_M = np.transpose(np.reshape(lin_reg.coef_, shape=(p, p)))
    frob = np.linalg.norm(est_M - true_M)

    estimate_2_1[seed-offset, 0] = est_M.round(3)
    frobnorm_2_1[seed-offset, 0] = frob

    for index, param in enumerate(reg_param):
        direct_lasso = linear_model.Lasso(alpha=param, max_iter=5000)
        direct_lasso.fit(X=emp_A, y=-c)
        est_M = np.transpose(np.reshape(direct_lasso.coef_, shape=(p, p)))
        frob = np.linalg.norm(est_M - true_M)

        estimate_2_1[seed-offset, index+1] = est_M.round(3)
        frobnorm_2_1[seed-offset, index+1] = frob

In [8]:
np.mean(frobnorm_2_1, axis=0)

array([np.float64(1.109647579854012), np.float64(1.0234657976362693),
       np.float64(1.098175096620818), np.float64(1.1259596413563264),
       np.float64(1.5344809765349823), np.float64(2.544526165275287)],
      dtype=object)

### $\texttt{index\_deletion = 2}$
***Results:***
* Average Frobenius distance between the estimation $\hat{M}$ and the true matrix $M^*$ for linear regression is $\large{1.834}$.
* Average Frobenius distance between the estimation $\hat{M}$ and the true matrix $M^*$ for Lasso regression with $\lambda = 0.0001$ is $\large{1.761}$.
* Average Frobenius distance between the estimation $\hat{M}$ and the true matrix $M^*$ for Lasso regression with $\lambda = 0.001$ is $\large{1.804}$.
* Average Frobenius distance between the estimation $\hat{M}$ and the true matrix $M^*$ for Lasso regression with $\lambda = 0.01$ is $\large{1.957}$.
* Average Frobenius distance between the estimation $\hat{M}$ and the true matrix $M^*$ for Lasso regression with $\lambda = 0.1$ is $\large{2.129}$.
* Average Frobenius distance between the estimation $\hat{M}$ and the true matrix $M^*$ for Lasso regression with $\lambda = 1$ is $\large{2.581}$.

In [9]:
index_deletion = 2
offset = 123

c = np.hstack((vech_C, - b * unit_v))
c = np.delete(c, (index_deletion))

estimate_2_2 = np.empty(shape=(num_exp, num_param), dtype=object)
frobnorm_2_2 = np.empty(shape=(num_exp, num_param), dtype=object)

for seed in range(offset, num_exp + offset):
    true_M = simulate_M(seed=seed, p=p)

    true_cov = solve_continuous_lyapunov(a=true_M, q=-C)
    true_mean = (b * np.linalg.inv(a=true_M)).dot(unit_v)

    obs = np.random.multivariate_normal(mean = true_mean, cov = true_cov, size = num_sim)
    emp_cov = np.cov(obs, rowvar=False)
    emp_mean = np.mean(obs, axis=0)

    emp_A_Sigma = create_A_Sigma(cov=emp_cov, p=p)
    emp_A = np.delete(create_A(A_Sigma=emp_A_Sigma, mean=emp_mean, p=p), (index_deletion), axis=0)

    # Linear regression
    lin_reg = linear_model.LinearRegression()
    lin_reg.fit(X=emp_A, y=-c)
    est_M = np.transpose(np.reshape(lin_reg.coef_, shape=(p, p)))
    frob = np.linalg.norm(est_M - true_M)

    estimate_2_2[seed-offset, 0] = est_M.round(3)
    frobnorm_2_2[seed-offset, 0] = frob

    for index, param in enumerate(reg_param):
        direct_lasso = linear_model.Lasso(alpha=param, max_iter=5000)
        direct_lasso.fit(X=emp_A, y=-c)
        est_M = np.transpose(np.reshape(direct_lasso.coef_, shape=(p, p)))
        frob = np.linalg.norm(est_M - true_M)

        estimate_2_2[seed-offset, index+1] = est_M.round(3)
        frobnorm_2_2[seed-offset, index+1] = frob

In [10]:
np.mean(frobnorm_2_2, axis=0)

array([np.float64(1.8341394399326245), np.float64(1.7617347116771944),
       np.float64(1.8040643901133049), np.float64(1.9567571459419715),
       np.float64(2.1297214900596284), np.float64(2.580841638732733)],
      dtype=object)

### $\texttt{index\_deletion = 3}$
***Results:***
* Average Frobenius distance between the estimation $\hat{M}$ and the true matrix $M^*$ for linear regression is $\large{1.173}$.
* Average Frobenius distance between the estimation $\hat{M}$ and the true matrix $M^*$ for Lasso regression with $\lambda = 0.0001$ is $\large{1.331}$.
* Average Frobenius distance between the estimation $\hat{M}$ and the true matrix $M^*$ for Lasso regression with $\lambda = 0.001$ is $\large{1.099}$.
* Average Frobenius distance between the estimation $\hat{M}$ and the true matrix $M^*$ for Lasso regression with $\lambda = 0.01$ is $\large{1.053}$.
* Average Frobenius distance between the estimation $\hat{M}$ and the true matrix $M^*$ for Lasso regression with $\lambda = 0.1$ is $\large{1.808}$.
* Average Frobenius distance between the estimation $\hat{M}$ and the true matrix $M^*$ for Lasso regression with $\lambda = 1$ is $\large{2.716}$.

In [11]:
index_deletion = 3
offset = 1234

c = np.hstack((vech_C, - b * unit_v))
c = np.delete(c, (index_deletion))

estimate_2_3 = np.empty(shape=(num_exp, num_param), dtype=object)
frobnorm_2_3 = np.empty(shape=(num_exp, num_param), dtype=object)

for seed in range(offset, num_exp + offset):
    true_M = simulate_M(seed=seed, p=p)

    true_cov = solve_continuous_lyapunov(a=true_M, q=-C)
    true_mean = (b * np.linalg.inv(a=true_M)).dot(unit_v)

    obs = np.random.multivariate_normal(mean = true_mean, cov = true_cov, size = num_sim)
    emp_cov = np.cov(obs, rowvar=False)
    emp_mean = np.mean(obs, axis=0)

    emp_A_Sigma = create_A_Sigma(cov=emp_cov, p=p)
    emp_A = np.delete(create_A(A_Sigma=emp_A_Sigma, mean=emp_mean, p=p), (index_deletion), axis=0)

    # Linear regression
    lin_reg = linear_model.LinearRegression()
    lin_reg.fit(X=emp_A, y=-c)
    est_M = np.transpose(np.reshape(lin_reg.coef_, shape=(p, p)))
    frob = np.linalg.norm(est_M - true_M)

    estimate_2_3[seed-offset, 0] = est_M.round(3)
    frobnorm_2_3[seed-offset, 0] = frob

    for index, param in enumerate(reg_param):
        direct_lasso = linear_model.Lasso(alpha=param, max_iter=5000)
        direct_lasso.fit(X=emp_A, y=-c)
        est_M = np.transpose(np.reshape(direct_lasso.coef_, shape=(p, p)))
        frob = np.linalg.norm(est_M - true_M)

        estimate_2_3[seed-offset, index+1] = est_M.round(3)
        frobnorm_2_3[seed-offset, index+1] = frob

In [12]:
np.mean(frobnorm_2_3, axis=0)

array([np.float64(1.173212079122521), np.float64(1.3314222377185199),
       np.float64(1.0992103591417925), np.float64(1.0527467957458996),
       np.float64(1.807932496222883), np.float64(2.7161311930542973)],
      dtype=object)

### $\texttt{index\_deletion = 4}$
***Results:***
* Average Frobenius distance between the estimation $\hat{M}$ and the true matrix $M^*$ for linear regression is $\large{0.525}$.
* Average Frobenius distance between the estimation $\hat{M}$ and the true matrix $M^*$ for Lasso regression with $\lambda = 0.0001$ is $\large{0.559}$.
* Average Frobenius distance between the estimation $\hat{M}$ and the true matrix $M^*$ for Lasso regression with $\lambda = 0.001$ is $\large{0.684}$.
* Average Frobenius distance between the estimation $\hat{M}$ and the true matrix $M^*$ for Lasso regression with $\lambda = 0.01$ is $\large{0.774}$.
* Average Frobenius distance between the estimation $\hat{M}$ and the true matrix $M^*$ for Lasso regression with $\lambda = 0.1$ is $\large{1.466}$.
* Average Frobenius distance between the estimation $\hat{M}$ and the true matrix $M^*$ for Lasso regression with $\lambda = 1$ is $\large{2.56}$.

In [13]:
index_deletion = 4
offset = 12345

c = np.hstack((vech_C, - b * unit_v))
c = np.delete(c, (index_deletion))

estimate_2_4 = np.empty(shape=(num_exp, num_param), dtype=object)
frobnorm_2_4 = np.empty(shape=(num_exp, num_param), dtype=object)

for seed in range(offset, num_exp + offset):
    true_M = simulate_M(seed=seed, p=p)

    true_cov = solve_continuous_lyapunov(a=true_M, q=-C)
    true_mean = (b * np.linalg.inv(a=true_M)).dot(unit_v)

    obs = np.random.multivariate_normal(mean = true_mean, cov = true_cov, size = num_sim)
    emp_cov = np.cov(obs, rowvar=False)
    emp_mean = np.mean(obs, axis=0)

    emp_A_Sigma = create_A_Sigma(cov=emp_cov, p=p)
    emp_A = np.delete(create_A(A_Sigma=emp_A_Sigma, mean=emp_mean, p=p), (index_deletion), axis=0)

    # Linear regression
    lin_reg = linear_model.LinearRegression()
    lin_reg.fit(X=emp_A, y=-c)
    est_M = np.transpose(np.reshape(lin_reg.coef_, shape=(p, p)))
    frob = np.linalg.norm(est_M - true_M)

    estimate_2_4[seed-offset, 0] = est_M.round(3)
    frobnorm_2_4[seed-offset, 0] = frob

    for index, param in enumerate(reg_param):
        direct_lasso = linear_model.Lasso(alpha=param, max_iter=5000)
        direct_lasso.fit(X=emp_A, y=-c)
        est_M = np.transpose(np.reshape(direct_lasso.coef_, shape=(p, p)))
        frob = np.linalg.norm(est_M - true_M)

        estimate_2_4[seed-offset, index+1] = est_M.round(3)
        frobnorm_2_4[seed-offset, index+1] = frob

In [14]:
np.mean(frobnorm_2_4, axis=0)

array([np.float64(0.5254359070100943), np.float64(0.5586466044393253),
       np.float64(0.6843382765957513), np.float64(0.7738859483887953),
       np.float64(1.4657694011316678), np.float64(2.5603087786860352)],
      dtype=object)

### No deletion
***Results:***
* Average Frobenius distance between the estimation $\hat{M}$ and the true matrix $M^*$ for linear regression is $\large{0.317}$.
* Average Frobenius distance between the estimation $\hat{M}$ and the true matrix $M^*$ for Lasso regression with $\lambda = 0.0001$ is $\large{0.269}$.
* Average Frobenius distance between the estimation $\hat{M}$ and the true matrix $M^*$ for Lasso regression with $\lambda = 0.001$ is $\large{0.257}$.
* Average Frobenius distance between the estimation $\hat{M}$ and the true matrix $M^*$ for Lasso regression with $\lambda = 0.01$ is $\large{0.435}$.
* Average Frobenius distance between the estimation $\hat{M}$ and the true matrix $M^*$ for Lasso regression with $\lambda = 0.1$ is $\large{1.485}$.
* Average Frobenius distance between the estimation $\hat{M}$ and the true matrix $M^*$ for Lasso regression with $\lambda = 1$ is $\large{2.615}$.

In [15]:
offset = 12345

c = np.hstack((vech_C, - b * unit_v))

estimate_2 = np.empty(shape=(num_exp, num_param), dtype=object)
frobnorm_2 = np.empty(shape=(num_exp, num_param), dtype=object)

for seed in range(offset, num_exp + offset):
    true_M = simulate_M(seed=seed, p=p)

    true_cov = solve_continuous_lyapunov(a=true_M, q=-C)
    true_mean = (b * np.linalg.inv(a=true_M)).dot(unit_v)

    obs = np.random.multivariate_normal(mean = true_mean, cov = true_cov, size = num_sim)
    emp_cov = np.cov(obs, rowvar=False)
    emp_mean = np.mean(obs, axis=0)

    emp_A_Sigma = create_A_Sigma(cov=emp_cov, p=p)
    emp_A = create_A(A_Sigma=emp_A_Sigma, mean=emp_mean, p=p)

    # Linear regression
    lin_reg = linear_model.LinearRegression()
    lin_reg.fit(X=emp_A, y=-c)
    est_M = np.transpose(np.reshape(lin_reg.coef_, shape=(p, p)))
    frob = np.linalg.norm(est_M - true_M)

    estimate_2[seed-offset, 0] = est_M.round(3)
    frobnorm_2[seed-offset, 0] = frob

    for index, param in enumerate(reg_param):
        direct_lasso = linear_model.Lasso(alpha=param, max_iter=5000)
        direct_lasso.fit(X=emp_A, y=-c)
        est_M = np.transpose(np.reshape(direct_lasso.coef_, shape=(p, p)))
        frob = np.linalg.norm(est_M - true_M)

        estimate_2[seed-offset, index+1] = est_M.round(3)
        frobnorm_2[seed-offset, index+1] = frob

In [16]:
np.mean(frobnorm_2, axis=0)

array([np.float64(0.31640154268575704), np.float64(0.2694951318754762),
       np.float64(0.25661614152081935), np.float64(0.4347665218428023),
       np.float64(1.4850007438856891), np.float64(2.6153284947527413)],
      dtype=object)

### Overview: 2 nodes
* Average Frobenius distance between the estimation $\hat{M}$ and the true matrix $M^*$ for linear regression is $\large{0.962}$.
* Average Frobenius distance between the estimation $\hat{M}$ and the true matrix $M^*$ for Lasso regression with $\lambda = 0.0001$ is $\large{0.996}$.
* Average Frobenius distance between the estimation $\hat{M}$ and the true matrix $M^*$ for Lasso regression with $\lambda = 0.001$ is $\large{0.985}$.
* Average Frobenius distance between the estimation $\hat{M}$ and the true matrix $M^*$ for Lasso regression with $\lambda = 0.01$ is $\large{1.052}$.
* Average Frobenius distance between the estimation $\hat{M}$ and the true matrix $M^*$ for Lasso regression with $\lambda = 0.1$ is $\large{1.653}$.
* Average Frobenius distance between the estimation $\hat{M}$ and the true matrix $M^*$ for Lasso regression with $\lambda = 1$ is $\large{2.61}$.

In [17]:
frobnorm_2nodes = np.vstack((frobnorm_2, frobnorm_2_0, frobnorm_2_1, frobnorm_2_2, frobnorm_2_3, frobnorm_2_4))
np.mean(frobnorm_2nodes, axis=0)

array([np.float64(0.9622928820203176), np.float64(0.9962117798858562),
       np.float64(0.9845901884369537), np.float64(1.051632707783403),
       np.float64(1.6534434267266764), np.float64(2.61034298207602)],
      dtype=object)

## 3 nodes

In [18]:
p = 3

C = 2 * np.eye(p)
vech_C = np.array([C[i, j] for i in range(p) for j in range(i, p)])
unit_v = np.eye(p)[:, index_intervention]
c = np.hstack((vech_C, - b * unit_v))

reg_param = [0.0001, 0.001, 0.005, 0.01, 0.05]
num_param = len(reg_param) + 1 # also linear regression

### All entries
***Results:***
* Average Frobenius distance between the estimation $\hat{M}$ and the true matrix $M^*$ for linear regression is $\large{1.599}$.
* Average Frobenius distance between the estimation $\hat{M}$ and the true matrix $M^*$ for Lasso regression with $\lambda = 0.0001$ is $\large{1.214}$.
* Average Frobenius distance between the estimation $\hat{M}$ and the true matrix $M^*$ for Lasso regression with $\lambda = 0.001$ is $\large{1.198}$.
* Average Frobenius distance between the estimation $\hat{M}$ and the true matrix $M^*$ for Lasso regression with $\lambda = 0.005$ is $\large{1.428}$.
* Average Frobenius distance between the estimation $\hat{M}$ and the true matrix $M^*$ for Lasso regression with $\lambda = 0.01$ is $\large{1.75}$.
* Average Frobenius distance between the estimation $\hat{M}$ and the true matrix $M^*$ for Lasso regression with $\lambda = 0.05$ is $\large{3.252}$.

In [19]:
offset = 120

estimate_3_0 = np.empty(shape=(num_exp, num_param), dtype=object)
frobnorm_3_0 = np.empty(shape=(num_exp, num_param), dtype=object)

for seed in range(offset, num_exp + offset):
    true_M = simulate_M(seed=seed, p=p)

    true_cov = solve_continuous_lyapunov(a=true_M, q=-C)
    true_mean = (b * np.linalg.inv(a=true_M)).dot(unit_v)

    obs = np.random.multivariate_normal(mean = true_mean, cov = true_cov, size = num_sim)
    emp_cov = np.cov(obs, rowvar=False)
    emp_mean = np.mean(obs, axis=0)

    emp_A_Sigma = create_A_Sigma(cov=emp_cov, p=p)
    emp_A = create_A(A_Sigma=emp_A_Sigma, mean=emp_mean, p=p)

    # Linear regression
    lin_reg = linear_model.LinearRegression()
    lin_reg.fit(X=emp_A, y=-c)
    est_M = np.transpose(np.reshape(lin_reg.coef_, shape=(p, p)))
    frob = np.linalg.norm(est_M - true_M)

    estimate_3_0[seed-offset, 0] = est_M.round(3)
    frobnorm_3_0[seed-offset, 0] = frob

    for index, param in enumerate(reg_param):
        direct_lasso = linear_model.Lasso(alpha=param, max_iter=5000)
        direct_lasso.fit(X=emp_A, y=-c)
        est_M = np.transpose(np.reshape(direct_lasso.coef_, shape=(p, p)))
        frob = np.linalg.norm(est_M - true_M)

        estimate_3_0[seed-offset, index+1] = est_M.round(3)
        frobnorm_3_0[seed-offset, index+1] = frob

In [20]:
np.mean(frobnorm_3_0, axis=0)

array([np.float64(1.5985822382812176), np.float64(1.213712848740723),
       np.float64(1.1979436840885103), np.float64(1.4276400870415886),
       np.float64(1.7503049194905516), np.float64(3.251963630348755)],
      dtype=object)

### One random entry missing
***Results:***
* Average Frobenius distance between the estimation $\hat{M}$ and the true matrix $M^*$ for linear regression is $\large{1.402}$.
* Average Frobenius distance between the estimation $\hat{M}$ and the true matrix $M^*$ for Lasso regression with $\lambda = 0.0001$ is $\large{1.165}$.
* Average Frobenius distance between the estimation $\hat{M}$ and the true matrix $M^*$ for Lasso regression with $\lambda = 0.001$ is $\large{1.16}$.
* Average Frobenius distance between the estimation $\hat{M}$ and the true matrix $M^*$ for Lasso regression with $\lambda = 0.005$ is $\large{1.393}$.
* Average Frobenius distance between the estimation $\hat{M}$ and the true matrix $M^*$ for Lasso regression with $\lambda = 0.01$ is $\large{1.722}$.
* Average Frobenius distance between the estimation $\hat{M}$ and the true matrix $M^*$ for Lasso regression with $\lambda = 0.05$ is $\large{3.236}$.

In [21]:
entries = [[0, 1], [0, 2], [1, 0], [1, 2], [2, 0], [2, 1]]
offset = 1208

estimate_3_1 = np.empty(shape=(num_exp, num_param), dtype=object)
frobnorm_3_1 = np.empty(shape=(num_exp, num_param), dtype=object)

for seed in range(offset, num_exp + offset):
    true_M = simulate_M(seed=seed, p=p)

    true_cov = solve_continuous_lyapunov(a=true_M, q=-C)
    true_mean = (b * np.linalg.inv(a=true_M)).dot(unit_v)

    obs = np.random.multivariate_normal(mean = true_mean, cov = true_cov, size = num_sim)
    emp_cov = np.cov(obs, rowvar=False)
    emp_mean = np.mean(obs, axis=0)

    emp_A_Sigma = create_A_Sigma(cov=emp_cov, p=p)
    emp_A = create_A(A_Sigma=emp_A_Sigma, mean=emp_mean, p=p)

    # Linear regression
    lin_reg = linear_model.LinearRegression()
    lin_reg.fit(X=emp_A, y=-c)
    est_M = np.transpose(np.reshape(lin_reg.coef_, shape=(p, p)))
    frob = np.linalg.norm(est_M - true_M)

    estimate_3_1[seed-offset, 0] = est_M.round(3)
    frobnorm_3_1[seed-offset, 0] = frob

    for index, param in enumerate(reg_param):
        direct_lasso = linear_model.Lasso(alpha=param, max_iter=5000)
        direct_lasso.fit(X=emp_A, y=-c)
        est_M = np.transpose(np.reshape(direct_lasso.coef_, shape=(p, p)))
        frob = np.linalg.norm(est_M - true_M)

        estimate_3_1[seed-offset, index+1] = est_M.round(3)
        frobnorm_3_1[seed-offset, index+1] = frob

In [22]:
np.mean(frobnorm_3_1, axis=0)

array([np.float64(1.4020310184075546), np.float64(1.1646270080806258),
       np.float64(1.1601020306999925), np.float64(1.3926530057673174),
       np.float64(1.7224551100129262), np.float64(3.2362609216277045)],
      dtype=object)

### Two entries missing I
***Results:***
* Average Frobenius distance between the estimation $\hat{M}$ and the true matrix $M^*$ for linear regression is $\large{1.469}$.
* Average Frobenius distance between the estimation $\hat{M}$ and the true matrix $M^*$ for Lasso regression with $\lambda = 0.0001$ is $\large{1.193}$.
* Average Frobenius distance between the estimation $\hat{M}$ and the true matrix $M^*$ for Lasso regression with $\lambda = 0.001$ is $\large{1.165}$.
* Average Frobenius distance between the estimation $\hat{M}$ and the true matrix $M^*$ for Lasso regression with $\lambda = 0.005$ is $\large{1.385}$.
* Average Frobenius distance between the estimation $\hat{M}$ and the true matrix $M^*$ for Lasso regression with $\lambda = 0.01$ is $\large{1.703}$.
* Average Frobenius distance between the estimation $\hat{M}$ and the true matrix $M^*$ for Lasso regression with $\lambda = 0.05$ is $\large{3.216}$.

In [23]:
offset = 12089

estimate_3_2 = np.empty(shape=(num_exp, num_param), dtype=object)
frobnorm_3_2 = np.empty(shape=(num_exp, num_param), dtype=object)

for seed in range(offset, num_exp + offset):
    true_M = simulate_M(seed=seed, p=p)

    true_cov = solve_continuous_lyapunov(a=true_M, q=-C)
    true_mean = (b * np.linalg.inv(a=true_M)).dot(unit_v)

    obs = np.random.multivariate_normal(mean = true_mean, cov = true_cov, size = num_sim)
    emp_cov = np.cov(obs, rowvar=False)
    emp_mean = np.mean(obs, axis=0)

    emp_A_Sigma = create_A_Sigma(cov=emp_cov, p=p)
    emp_A = create_A(A_Sigma=emp_A_Sigma, mean=emp_mean, p=p)

    # Linear regression
    lin_reg = linear_model.LinearRegression()
    lin_reg.fit(X=emp_A, y=-c)
    est_M = np.transpose(np.reshape(lin_reg.coef_, shape=(p, p)))
    frob = np.linalg.norm(est_M - true_M)

    estimate_3_2[seed-offset, 0] = est_M.round(3)
    frobnorm_3_2[seed-offset, 0] = frob

    for index, param in enumerate(reg_param):
        direct_lasso = linear_model.Lasso(alpha=param, max_iter=5000)
        direct_lasso.fit(X=emp_A, y=-c)
        est_M = np.transpose(np.reshape(direct_lasso.coef_, shape=(p, p)))
        frob = np.linalg.norm(est_M - true_M)

        estimate_3_2[seed-offset, index+1] = est_M.round(3)
        frobnorm_3_2[seed-offset, index+1] = frob

In [24]:
np.mean(frobnorm_3_2, axis=0)

array([np.float64(1.468500549131823), np.float64(1.1925710057889076),
       np.float64(1.1649036826929196), np.float64(1.3847339229662154),
       np.float64(1.7029778835314224), np.float64(3.2156604401133935)],
      dtype=object)

### Two entries missing II
***Results:***
* Average Frobenius distance between the estimation $\hat{M}$ and the true matrix $M^*$ for linear regression is $\large{1.494}$.
* Average Frobenius distance between the estimation $\hat{M}$ and the true matrix $M^*$ for Lasso regression with $\lambda = 0.0001$ is $\large{1.207}$.
* Average Frobenius distance between the estimation $\hat{M}$ and the true matrix $M^*$ for Lasso regression with $\lambda = 0.001$ is $\large{1.165}$.
* Average Frobenius distance between the estimation $\hat{M}$ and the true matrix $M^*$ for Lasso regression with $\lambda = 0.005$ is $\large{1.362}$.
* Average Frobenius distance between the estimation $\hat{M}$ and the true matrix $M^*$ for Lasso regression with $\lambda = 0.01$ is $\large{1.677}$.
* Average Frobenius distance between the estimation $\hat{M}$ and the true matrix $M^*$ for Lasso regression with $\lambda = 0.05$ is $\large{3.201}$.

In [25]:
offset = 120899

estimate_3_3 = np.empty(shape=(num_exp, num_param), dtype=object)
frobnorm_3_3 = np.empty(shape=(num_exp, num_param), dtype=object)

for seed in range(offset, num_exp + offset):
    true_M = simulate_M(seed=seed, p=p)

    true_cov = solve_continuous_lyapunov(a=true_M, q=-C)
    true_mean = (b * np.linalg.inv(a=true_M)).dot(unit_v)

    obs = np.random.multivariate_normal(mean = true_mean, cov = true_cov, size = num_sim)
    emp_cov = np.cov(obs, rowvar=False)
    emp_mean = np.mean(obs, axis=0)

    emp_A_Sigma = create_A_Sigma(cov=emp_cov, p=p)
    emp_A = create_A(A_Sigma=emp_A_Sigma, mean=emp_mean, p=p)

    # Linear regression
    lin_reg = linear_model.LinearRegression()
    lin_reg.fit(X=emp_A, y=-c)
    est_M = np.transpose(np.reshape(lin_reg.coef_, shape=(p, p)))
    frob = np.linalg.norm(est_M - true_M)

    estimate_3_3[seed-offset, 0] = est_M.round(3)
    frobnorm_3_3[seed-offset, 0] = frob

    for index, param in enumerate(reg_param):
        direct_lasso = linear_model.Lasso(alpha=param, max_iter=5000)
        direct_lasso.fit(X=emp_A, y=-c)
        est_M = np.transpose(np.reshape(direct_lasso.coef_, shape=(p, p)))
        frob = np.linalg.norm(est_M - true_M)

        estimate_3_3[seed-offset, index+1] = est_M.round(3)
        frobnorm_3_3[seed-offset, index+1] = frob

In [26]:
np.mean(frobnorm_3_3, axis=0)

array([np.float64(1.494461891642084), np.float64(1.207229674223321),
       np.float64(1.1654669389367935), np.float64(1.3617718110742767),
       np.float64(1.6774434061129226), np.float64(3.201126109718212)],
      dtype=object)

### Overview: 3 nodes
* Average Frobenius distance between the estimation $\hat{M}$ and the true matrix $M^*$ for linear regression is $\large{1.491}$.
* Average Frobenius distance between the estimation $\hat{M}$ and the true matrix $M^*$ for Lasso regression with $\lambda = 0.0001$ is $\large{1.195}$.
* Average Frobenius distance between the estimation $\hat{M}$ and the true matrix $M^*$ for Lasso regression with $\lambda = 0.001$ is $\large{1.172}$.
* Average Frobenius distance between the estimation $\hat{M}$ and the true matrix $M^*$ for Lasso regression with $\lambda = 0.005$ is $\large{1.392}$.
* Average Frobenius distance between the estimation $\hat{M}$ and the true matrix $M^*$ for Lasso regression with $\lambda = 0.01$ is $\large{1.713}$.
* Average Frobenius distance between the estimation $\hat{M}$ and the true matrix $M^*$ for Lasso regression with $\lambda = 0.05$ is $\large{3.226}$.

In [27]:
frobnorm_3nodes = np.vstack((frobnorm_3_0, frobnorm_3_1, frobnorm_3_2, frobnorm_3_3))
np.mean(frobnorm_3nodes, axis=0)

array([np.float64(1.4908939243656711), np.float64(1.1945351342083956),
       np.float64(1.172104084104554), np.float64(1.3916997067123489),
       np.float64(1.713295329786958), np.float64(3.226252775452022)],
      dtype=object)

## 4 nodes
***Results:***
* Average Frobenius distance between the estimation $\hat{M}$ and the true matrix $M^*$ for linear regression is $\large{2.407}$.
* Average Frobenius distance between the estimation $\hat{M}$ and the true matrix $M^*$ for Lasso regression with $\lambda = 0.0001$ is $\large{2.108}$.
* Average Frobenius distance between the estimation $\hat{M}$ and the true matrix $M^*$ for Lasso regression with $\lambda = 0.001$ is $\large{2.086}$.
* Average Frobenius distance between the estimation $\hat{M}$ and the true matrix $M^*$ for Lasso regression with $\lambda = 0.005$ is $\large{2.76}$.
* Average Frobenius distance between the estimation $\hat{M}$ and the true matrix $M^*$ for Lasso regression with $\lambda = 0.01$ is $\large{3.588}$.
* Average Frobenius distance between the estimation $\hat{M}$ and the true matrix $M^*$ for Lasso regression with $\lambda = 0.05$ is $\large{6.067}$.

In [28]:
p = 4

C = 2 * np.eye(p)
vech_C = np.array([C[i, j] for i in range(p) for j in range(i, p)])
unit_v = np.eye(p)[:, index_intervention]
c = np.hstack((vech_C, - b * unit_v))

reg_param = [0.0001, 0.001, 0.005, 0.01, 0.05]
num_param = len(reg_param) + 1 # also linear regression
offset = 240

estimate_4 = np.empty(shape=(num_exp, num_param), dtype=object)
frobnorm_4 = np.empty(shape=(num_exp, num_param), dtype=object)

for seed in range(offset, num_exp + offset):
    true_M = simulate_M(seed=seed, p=p)

    true_cov = solve_continuous_lyapunov(a=true_M, q=-C)
    true_mean = (b * np.linalg.inv(a=true_M)).dot(unit_v)

    obs = np.random.multivariate_normal(mean = true_mean, cov = true_cov, size = num_sim)
    emp_cov = np.cov(obs, rowvar=False)
    emp_mean = np.mean(obs, axis=0)

    emp_A_Sigma = create_A_Sigma(cov=emp_cov, p=p)
    emp_A = create_A(A_Sigma=emp_A_Sigma, mean=emp_mean, p=p)

    # Linear regression
    lin_reg = linear_model.LinearRegression()
    lin_reg.fit(X=emp_A, y=-c)
    est_M = np.transpose(np.reshape(lin_reg.coef_, shape=(p, p)))
    frob = np.linalg.norm(est_M - true_M)

    estimate_4[seed-offset, 0] = est_M.round(3)
    frobnorm_4[seed-offset, 0] = frob

    for index, param in enumerate(reg_param):
        direct_lasso = linear_model.Lasso(alpha=param, max_iter=5000)
        direct_lasso.fit(X=emp_A, y=-c)
        est_M = np.transpose(np.reshape(direct_lasso.coef_, shape=(p, p)))
        frob = np.linalg.norm(est_M - true_M)

        estimate_4[seed-offset, index+1] = est_M.round(3)
        frobnorm_4[seed-offset, index+1] = frob

In [29]:
np.mean(frobnorm_4, axis=0)

array([np.float64(2.4072481489133537), np.float64(2.1084008205003952),
       np.float64(2.0861951798924325), np.float64(2.7598147910651525),
       np.float64(3.588047584401723), np.float64(6.066892127435979)],
      dtype=object)

## 5 nodes
***Results:***
* Average Frobenius distance between the estimation $\hat{M}$ and the true matrix $M^*$ for linear regression is $\large{3.26}$.
* Average Frobenius distance between the estimation $\hat{M}$ and the true matrix $M^*$ for Lasso regression with $\lambda = 0.0001$ is $\large{3.047}$.
* Average Frobenius distance between the estimation $\hat{M}$ and the true matrix $M^*$ for Lasso regression with $\lambda = 0.001$ is $\large{3.095}$.
* Average Frobenius distance between the estimation $\hat{M}$ and the true matrix $M^*$ for Lasso regression with $\lambda = 0.005$ is $\large{4.587}$.
* Average Frobenius distance between the estimation $\hat{M}$ and the true matrix $M^*$ for Lasso regression with $\lambda = 0.01$ is $\large{5.743}$.
* Average Frobenius distance between the estimation $\hat{M}$ and the true matrix $M^*$ for Lasso regression with $\lambda = 0.05$ is $\large{9.537}$.

In [30]:
p = 5

C = 2 * np.eye(p)
vech_C = np.array([C[i, j] for i in range(p) for j in range(i, p)])
unit_v = np.eye(p)[:, index_intervention]
c = np.hstack((vech_C, - b * unit_v))

reg_param = [0.0001, 0.001, 0.005, 0.01, 0.05]
num_param = len(reg_param) + 1 # also linear regression
offset = 2409

estimate_5 = np.empty(shape=(num_exp, num_param), dtype=object)
frobnorm_5 = np.empty(shape=(num_exp, num_param), dtype=object)

for seed in range(offset, num_exp + offset):
    true_M = simulate_M(seed=seed, p=p)

    true_cov = solve_continuous_lyapunov(a=true_M, q=-C)
    true_mean = (b * np.linalg.inv(a=true_M)).dot(unit_v)

    obs = np.random.multivariate_normal(mean = true_mean, cov = true_cov, size = num_sim)
    emp_cov = np.cov(obs, rowvar=False)
    emp_mean = np.mean(obs, axis=0)

    emp_A_Sigma = create_A_Sigma(cov=emp_cov, p=p)
    emp_A = create_A(A_Sigma=emp_A_Sigma, mean=emp_mean, p=p)

    # Linear regression
    lin_reg = linear_model.LinearRegression()
    lin_reg.fit(X=emp_A, y=-c)
    est_M = np.transpose(np.reshape(lin_reg.coef_, shape=(p, p)))
    frob = np.linalg.norm(est_M - true_M)

    estimate_5[seed-offset, 0] = est_M.round(3)
    frobnorm_5[seed-offset, 0] = frob

    for index, param in enumerate(reg_param):
        direct_lasso = linear_model.Lasso(alpha=param, max_iter=5000)
        direct_lasso.fit(X=emp_A, y=-c)
        est_M = np.transpose(np.reshape(direct_lasso.coef_, shape=(p, p)))
        frob = np.linalg.norm(est_M - true_M)

        estimate_5[seed-offset, index+1] = est_M.round(3)
        frobnorm_5[seed-offset, index+1] = frob

In [31]:
np.mean(frobnorm_5, axis=0)

array([np.float64(3.2603465863331853), np.float64(3.0467216915159754),
       np.float64(3.0946854496177174), np.float64(4.586998208772058),
       np.float64(5.743267446759196), np.float64(9.536728044426047)],
      dtype=object)